# Silver → Gold Layer (Spark NLP)

Enrichissement des données Silver via Spark NLP : Sentiment Analysis, NER, Keyword Extraction.

## 1. Configuration Spark + Spark NLP

In [ ]:
from pyspark.sql import SparkSession

# TODO: A remplacer par vos propres identifiants Garage
GARAGE_ENDPOINT = "http://garage:3900"
GARAGE_ACCESS_KEY = "GK2ae23cad2bbbf648143b1b8c"
GARAGE_SECRET_KEY = "997e31832cbc9c78a2d919897f1cc9d63ad2c628464a7fba3a55f972c31790ee"

SILVER_PATH = "s3a://silver/hackernews"
GOLD_PATH = "s3a://gold/hackernews"

# TODO: Configurer volume partagé pour utiliser le cluster spark://spark:7077
# Mode local pour Spark NLP (les modèles sont téléchargés localement)
spark = SparkSession.builder \
    .appName("SilverToGold-SparkNLP") \
    .master("local[*]") \
    .config("spark.jars.packages", 
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.262,"
            "io.delta:delta-spark_2.12:3.3.0,"
            "com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.multiobjectdelete.enable", "false") \
    .config("spark.sql.shuffle.partitions", "10") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

hadoop_conf = spark.sparkContext._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.endpoint", GARAGE_ENDPOINT)
hadoop_conf.set("fs.s3a.access.key", GARAGE_ACCESS_KEY)
hadoop_conf.set("fs.s3a.secret.key", GARAGE_SECRET_KEY)
hadoop_conf.set("fs.s3a.endpoint.region", "garage")
hadoop_conf.set("fs.s3a.path.style.access", "true")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("fs.s3a.connection.ssl.enabled", "false")

In [ ]:
import sparknlp
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import (
    Tokenizer, Normalizer, StopWordsCleaner, LemmatizerModel,
    SentimentDLModel, NerDLModel, NerConverter,
    SentenceDetector, WordEmbeddingsModel, UniversalSentenceEncoder
)
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, explode, explode_outer, length, desc, count, when, rank, round as round_, window
from pyspark.sql.functions import sum as sum_
from pyspark.sql.window import Window

print(f"Spark NLP version: {sparknlp.version()}")

## 2. Lecture Silver

In [ ]:
comments_silver = spark.read.format("delta").load(f"{SILVER_PATH}/comments")
stories_silver = spark.read.format("delta").load(f"{SILVER_PATH}/stories")

print(f"Comments: {comments_silver.count()}, Stories: {stories_silver.count()}")

## 3. Sentiment Analysis

Classifier chaque commentaire comme positif ou négatif. USE (Universal Sentence Encoder) produit directement des sentence embeddings.

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("text_clean") \
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained("tfhub_use", "en") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence_embeddings")

sentiment_model = SentimentDLModel.pretrained("sentimentdl_use_imdb", "en") \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("sentiment")

sentiment_pipeline = Pipeline(stages=[
    document_assembler,
    use,
    sentiment_model
])

In [ ]:
sentiment_model_fitted = sentiment_pipeline.fit(comments_silver)
comments_with_sentiment = sentiment_model_fitted.transform(comments_silver)

In [ ]:
comments_sentiment = comments_with_sentiment \
    .withColumn("sentiment_result", explode(col("sentiment.result"))) \
    .select("id", "by", "parent", "text_clean", "timestamp", "sentiment_result")

comments_sentiment.show(5, truncate=50)

## 4. NER (Named Entity Recognition)

Extraire les entités nommées : personnes (PER), organisations (ORG), lieux (LOC). NER nécessite des word embeddings (GloVe).

In [ ]:
sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

word_embeddings = WordEmbeddingsModel.pretrained("glove_100d", "en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

ner_model = NerDLModel.pretrained("ner_dl", "en") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverter() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("entities")

ner_pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    word_embeddings,
    ner_model,
    ner_converter
])

In [ ]:
ner_model_fitted = ner_pipeline.fit(comments_silver)
comments_with_ner = ner_model_fitted.transform(comments_silver)

In [ ]:
comments_entities = comments_with_ner \
    .withColumn("entity", explode_outer(col("entities"))) \
    .select(
        "id", "by", "text_clean",
        col("entity.result").alias("entity_text"),
        col("entity.metadata.entity").alias("entity_type")
    )

comments_entities.filter(col("entity_text").isNotNull()).show(10, truncate=40)

## 5. Keyword Extraction

Identifier les mots-clés fréquents. Normalisation + suppression stop words + lemmatisation pour que "Running" et "run" soient comptés ensemble.

In [ ]:
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized") \
    .setLowercase(True)

stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(["normalized"]) \
    .setOutputCol("cleanTokens") \
    .setCaseSensitive(False)

lemmatizer = LemmatizerModel.pretrained("lemma_antbnc", "en") \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")

finisher = Finisher() \
    .setInputCols(["lemma"]) \
    .setOutputCols(["keywords"]) \
    .setOutputAsArray(True)

keywords_pipeline = Pipeline(stages=[
    document_assembler,
    sentence_detector,
    tokenizer,
    normalizer,
    stopwords_cleaner,
    lemmatizer,
    finisher
])

In [ ]:
keywords_model_fitted = keywords_pipeline.fit(comments_silver)
comments_with_keywords = keywords_model_fitted.transform(comments_silver)

In [ ]:
keywords_exploded = comments_with_keywords \
    .select("id", explode(col("keywords")).alias("keyword")) \
    .filter(length(col("keyword")) > 2)

top_keywords = keywords_exploded \
    .groupBy("keyword") \
    .count() \
    .orderBy(desc("count")) \
    .limit(50)

top_keywords.show(20)

## 6. Requêtes en batch

### 6a. SparkSQL - Sentiment par domaine



Jointure comments + stories pour répondre à : *"Quels sites génèrent les discussions les plus positives/négatives ?"*

In [ ]:
comments_sentiment.createOrReplaceTempView("comments_sentiment")
stories_silver.createOrReplaceTempView("stories")

sentiment_by_domain = spark.sql("""
    SELECT 
        s.domain,
        COUNT(*) as comment_count,
        SUM(CASE WHEN c.sentiment_result = 'pos' THEN 1 ELSE 0 END) as positive,
        SUM(CASE WHEN c.sentiment_result = 'neg' THEN 1 ELSE 0 END) as negative,
        ROUND(SUM(CASE WHEN c.sentiment_result = 'pos' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as positive_pct
    FROM comments_sentiment c
    JOIN stories s ON c.parent = s.id
    WHERE s.domain != ''
    GROUP BY s.domain
    HAVING COUNT(*) >= 5
    ORDER BY comment_count DESC
    LIMIT 20
""")

sentiment_by_domain.show(20, truncate=False)

### 6b. Window Function - Classement des auteurs

Utilisation de RANK() pour classer les auteurs par leur ratio de commentaires positifs.

In [ ]:
# Agrégation par auteur : total comments, positifs, négatifs
authors_stats = comments_sentiment \
    .groupBy("by") \
    .agg(
        count("*").alias("total_comments"),
        sum_(when(col("sentiment_result") == "pos", 1).otherwise(0)).alias("positive_count"),
        sum_(when(col("sentiment_result") == "neg", 1).otherwise(0)).alias("negative_count")
    ) \
    .filter(col("total_comments") >= 3) \
    .withColumn("positive_ratio", round_(col("positive_count") * 100.0 / col("total_comments"), 2))

# Window function : RANK par ratio de positivité
window_spec = Window.orderBy(desc("positive_ratio"), desc("total_comments"))

authors_ranked = authors_stats \
    .withColumn("rank", rank().over(window_spec)) \
    .filter(col("rank") <= 20)

authors_ranked.show(20, truncate=False)


## 7. Requêtes en streaming

In [ ]:
comments_stream = spark.readStream.format("delta").load(f"{SILVER_PATH}/comments")
comments_with_sentiment = sentiment_model_fitted.transform(comments_stream)                                                                                                                  

### 7a. Sentiment en temps réel

Agrégation du sentiment des commentaires par fenêtre de 5minutes:                                                                                                                                            

In [ ]:
sentiment_windowed = comments_with_sentiment \
    .withColumn("sentiment_result", explode(col("sentiment.result"))) \
    .withColumn("timestamp", col("timestamp").cast("timestamp")) \
    .withWatermark("timestamp", "10 minutes") \
    .groupBy(
        window(
            col("timestamp"),
            "5 minutes",
        )
    ) \
    .agg(
        count("*").alias("total_comments"),
        sum_(when(col("sentiment_result") == "pos", 1).otherwise(0)).alias("positive_count"),
        sum_(when(col("sentiment_result") == "neg", 1).otherwise(0)).alias("negative_count")
    ) \
    .withColumn("positive_ratio", round_(col("positive_count") * 100.0 / col("total_comments"), 2))

sentiment_windowed.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", f"{GOLD_PATH}/_checkpoints/sentiment_windowed") \
    .start(f"{GOLD_PATH}/sentiment_real_time")

Top mots-clés avec fenêtre glissante de 10 min, mise à jour toutes les 2 min:

In [ ]:
comments_with_kw = keywords_model_fitted.transform(comments_stream)
keywords_exploded_stream = comments_with_kw \
    .select(
        col("id"),
        col("timestamp"),
        explode(col("keywords")).alias("keyword")
    ) \
    .withColumn("timestamp", col("timestamp").cast("timestamp")) \
    .groupBy(
        window(
            col("timestamp"),
            "10 minutes",
            "2 minutes"
        ),
        col("keyword")
    ) \
    .count()

keywords_exploded_stream.writeStream \
    .format("delta") \
    .outputMode("complete") \
    .option("checkpointLocation", f"{GOLD_PATH}/_checkpoints/keywords_windowed") \
    .start(f"{GOLD_PATH}/keywords_real_time")

## 8. Visualisation Pandas + Seaborn

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sentiment_df = comments_sentiment.groupBy("sentiment_result").count().toPandas()

plt.figure(figsize=(8, 6))
sns.barplot(data=sentiment_df, x="sentiment_result", y="count", palette="viridis")
plt.title("Distribution des sentiments dans les commentaires HackerNews")
plt.xlabel("Sentiment")
plt.ylabel("Nombre de commentaires")
plt.tight_layout()
plt.show()

In [ ]:
top_entities_df = comments_entities \
    .filter(col("entity_type").isin(["ORG", "PRODUCT", "PERSON"])) \
    .groupBy("entity_text", "entity_type") \
    .count() \
    .orderBy(desc("count")) \
    .limit(15) \
    .toPandas()

plt.figure(figsize=(10, 6))
sns.barplot(data=top_entities_df, x="count", y="entity_text", hue="entity_type", dodge=False)
plt.title("Top entités mentionnées dans les commentaires HackerNews")
plt.xlabel("Nombre de mentions")
plt.ylabel("Entité")
plt.tight_layout()
plt.show()

In [ ]:
top_keywords_df = top_keywords.limit(20).toPandas()

plt.figure(figsize=(10, 6))
sns.barplot(data=top_keywords_df, x="count", y="keyword", palette="magma")
plt.title("Top 20 mots-clés dans les commentaires HackerNews")
plt.xlabel("Fréquence")
plt.ylabel("Mot-clé")
plt.tight_layout()
plt.savefig("top_keywords.png")
plt.show()

## 9. Écriture Gold

In [ ]:
comments_sentiment.write.format("delta").mode("overwrite").save(f"{GOLD_PATH}/comments_sentiment")

In [ ]:
entities_aggregated = comments_entities \
    .filter(col("entity_text").isNotNull()) \
    .groupBy("entity_text", "entity_type") \
    .count() \
    .orderBy(desc("count"))

entities_aggregated.write.format("delta").mode("overwrite").save(f"{GOLD_PATH}/entities")

In [ ]:
top_keywords.write.format("delta").mode("overwrite").save(f"{GOLD_PATH}/keywords")

In [ ]:
sentiment_by_domain.write.format("delta").mode("overwrite").save(f"{GOLD_PATH}/sentiment_by_domain")

## 10. Vérification

In [ ]:
spark.read.format("delta").load(f"{GOLD_PATH}/comments_sentiment").show(5, truncate=40)
spark.read.format("delta").load(f"{GOLD_PATH}/entities").show(10)
spark.read.format("delta").load(f"{GOLD_PATH}/keywords").show(10)
spark.read.format("delta").load(f"{GOLD_PATH}/sentiment_by_domain").show(10)

In [ ]:
spark.stop()